In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie
using LinearAlgebra, LinearSolve

cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305

t = 1000.0; tt = 1000.0; tstep = 0.001; tspan = [0.0, t]
integ_set = (alg = Vern9(), adaptive = false, dt = tstep);

In [3]:
index_hom = 1;

In [4]:
I0 = I0_hom[index_hom]; U0 = u0_hom[index_hom]
p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [11.325905642223786, 0.6594706953104683, 0.4863175026548461];

In [5]:
ds = CoupledODEs(TM, u0, p, diffeq = integ_set)
fp, _, _ = fixedpoints(ds, box, jacob_TM_);

In [6]:
ϵ_box = 1e-4
Ju0 = jacob_TM_(fp[1], p, 0);
fp_ = fp[1]

eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors

v1 = real(eigen_vectors[:, 1])
v2 = real(eigen_vectors[:, 2])
v3 = imag(eigen_vectors[:, 3])

v1 = normalize(v1, 2)
v2 = normalize(v2, 2)
v3 = normalize(v3, 2)

v1t = transpose(v1)
v2t = transpose(v2)
v3t = transpose(v3)

A = transpose([v1t; v2t; v3t])
A = Matrix(A)

3×3 Matrix{Float64}:
  0.998728    0.999899     0.0
 -0.0489879  -0.0141814    0.984697
  0.0119121   0.000705852  0.174274

In [62]:
N = 1001
nrange = range(1, N - 1, step = 1)
dots = zeros(length(nrange), 3)
data = zeros(length(nrange), 7);

In [63]:
for (i, n) in enumerate(nrange)
    α1 = 0
    α2 = cos(2 * pi * n / N) * (ϵ_box / 2)
    α3 = sin(2 * pi * n / N) * (ϵ_box / 2)
    αs = [α1, α2, α3]
    dot = fp_ + A * αs
    dots[i, :] = dot
end

In [65]:
function condition(u, t, integrator)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf) - ϵ_box
end
affect!(integrator) = terminate!(integrator)
function condition(u)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf), linsolve
end

condition (generic function with 2 methods)

In [66]:
cb = ContinuousCallback(condition, nothing, affect!)

for index in range(1, length(nrange), step = 1)
    
    u0 = SA[dots[index, 1], dots[index, 2], dots[index, 3]]
    prob = ODEProblem(TM, u0, tspan, p)
    sol = solve(prob, alg = RK4(), abstol = 1e-8, reltol = 1e-8, callback = cb)
    norm, linsolve = condition(sol[end])

    data[index, 1] = sol[1, end]
    data[index, 2] = sol[2, end]
    data[index, 3] = sol[3, end]
    data[index, 4:6] = linsolve
    data[index, 7] = norm
end

In [ ]:
idx, idy, idz = 2, 3, 1

f = Figure(resolution = (700, 700))

ax = LScene(f[1, 1], show_axis = true)
    
scale!(ax.scene, 50, 50, 1)

lines!(data[:, idx], data[:, idy], data[:, idz], linewidth = 1.0, color = :deeppink)

scatter!(ax, fp[1][idx], fp[1][idy], fp[1][idz], color = :blue)



display(GLMakie.Screen(), f);

In [73]:
f = Figure(resolution = (700, 700))

ax = Axis(f[1, 1])
idx, idy = 5, 6

scatter!(ax, 0.0, 0.0, color = :green, markersize = 15, marker = :rect)

scatter!(ax, data[:, idx], data[:, idy], color = :black, markersize = 5)


display(GLMakie.Screen(), f);